In [1]:
import sklearn
from efficientnet.keras import preprocess_input
import os
import librosa
import librosa.display
from lime.lime_tabular import LimeTabularExplainer
import pandas as pd
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from tensorflow.python.keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.callbacks import ModelCheckpoint
from datetime import datetime 
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.python.keras.optimizers import adam_v2
from sklearn import metrics
from tensorflow.python.keras.callbacks import ModelCheckpoint
from datetime import datetime 


In [2]:
trainpath= ("../reduce_sound/")

# get the training classes
keys=os.listdir(trainpath)
values=range(len(keys))

training_classes = dict (zip(keys,values))
print('training_classes = ',training_classes)

training_classes =  {'grey-cheeked warbler': 0, 'grey-sided laughingthrush': 1, 'grey-winged blackbird': 2, 'himalayan cuckoo': 3, 'indian spot-billed duck': 4, 'mountain scops owl': 5, 'mountain tailorbird': 6, 'pale blue flycatcher': 7, 'scaly laughingthrush': 8}


In [3]:
excel_data=pd.read_excel("sample_data_part.xlsx")
print(excel_data)

           mfcc0      mfcc1      mfcc2      mfcc3      mfcc4      mfcc5  \
0    -472.100769  58.775963   7.681985  23.957186   0.592718  16.293337   
1    -207.694870  77.358192 -61.414059 -14.969862 -31.636604  -9.872795   
2    -261.329163  29.816236 -31.988003   1.462996 -49.974438  11.225252   
3    -384.289886  52.660557   3.222627  43.804008 -44.171955  39.599606   
4    -349.680603  39.620228 -27.916094  71.358826   9.009825  13.436577   
...          ...        ...        ...        ...        ...        ...   
1345 -479.472260  29.047907   7.356060  63.971088 -15.386018  19.457771   
1346 -427.818329  74.157700  -1.176452  21.496334   0.987784  -2.916337   
1347 -471.515503  32.497757  -8.082848  18.765202  -0.467873  -0.629512   
1348 -489.081024  -5.154406 -75.384186  12.327448  18.345062  -1.690116   
1349 -446.537720  38.995647 -18.042435  27.678038   3.618870  -6.509332   

          mfcc6      mfcc7      mfcc8      mfcc9  ...    mfcc31    mfcc32  \
0      2.327442   7.68

In [4]:
#feature_length=len(extracted_features_df.columns)-1
#feature_length
feature_length=len(excel_data.columns)-1
#feature_list = extracted_features_df.columns[:feature_length].tolist()
feature_list = excel_data.columns[:feature_length].tolist()

#feature DataSet
#x=extracted_features_df[feature_list]
x=excel_data[feature_list]

#target DataSet
#y=extracted_features_df[feature_length]
y=excel_data[feature_length]

In [5]:
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))
y

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [6]:
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
print(X_train.shape)
#X_train.to_csv(index=False)
#X_test.to_csv(index=False)

(1080, 40)


In [7]:
num_labels=y.shape[1]
#Dense()
model=Sequential()
###first layer
model.add(Dense(200,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.1))
###second layer
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###third layer
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.2))
#fourth layer
model.add(Dense(400))
model.add(Activation('relu'))
model.add(Dropout(0.1))
###final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

2022-07-18 22:22:09.198099: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-07-18 22:22:09.198176: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (hrithik): /proc/driver/nvidia/version does not exist
2022-07-18 22:22:09.200058: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 200)               8200      
_________________________________________________________________
activation (Activation)      (None, 200)               0         
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               20100     
_________________________________________________________________
activation_1 (Activation)    (None, 100)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 200)               2

In [9]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='Adagrad')
num_epochs =220
num_batch_size =22

checkpointer = ModelCheckpoint(filepath='audio_classification.hdf5',
    verbose=1,
    save_best_only=True)
start = datetime.now()

history= model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/220
50/50 [==============================] - 1s 10ms/step - loss: 25.3326 - accuracy: 0.1056 - val_loss: 11.9983 - val_accuracy: 0.1333

Epoch 00001: val_loss improved from inf to 11.99834, saving model to audio_classification.hdf5
Epoch 2/220
50/50 [==============================] - 0s 5ms/step - loss: 25.2367 - accuracy: 0.1074 - val_loss: 11.7076 - val_accuracy: 0.1370

Epoch 00002: val_loss improved from 11.99834 to 11.70761, saving model to audio_classification.hdf5
Epoch 3/220
50/50 [==============================] - 0s 6ms/step - loss: 24.6683 - accuracy: 0.0991 - val_loss: 11.4794 - val_accuracy: 0.1296

Epoch 00003: val_loss improved from 11.70761 to 11.47942, saving model to audio_classification.hdf5
Epoch 4/220
50/50 [==============================] - 0s 6ms/step - loss: 23.8162 - accuracy: 0.1065 - val_loss: 11.1973 - val_accuracy: 0.1259

Epoch 00004: val_loss improved from 11.47942 to 11.19732, saving model to audio_classification.hdf5
Epoch 5/220
50/50 [=========

In [10]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print('test loss:{} \ntest acc:{}'.format(test_accuracy[0],test_accuracy[1]))

test loss:3.1902623176574707 
test acc:0.11481481790542603
